## Customer Churn Prediction using ANN

In [64]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [65]:
## Load the dataset
df = pd.read_csv('/Users/tajamulk2/Desktop/Customer-Churn-with-ANN/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Data Pre Processing

In [66]:
# Drop unnecessary columns
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [67]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


#### Encoding

In [68]:
# Convert categorical variables to numerical - Encoding
label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])

In [69]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [70]:
# One-hot encoding for Geography
from sklearn.preprocessing import OneHotEncoder
ohe_geo = OneHotEncoder()
geography_encoded = ohe_geo.fit_transform(df[['Geography']])
geography_df = pd.DataFrame(geography_encoded.toarray(), columns=ohe_geo.get_feature_names_out(['Geography']))
df = pd.concat([df, geography_df], axis=1)
df.drop('Geography', axis=1, inplace=True) 
df.head()


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


#### Create Pickle file to save Encoders

In [71]:
# Save the label encoder for Gender
with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)

# Save the one-hot encoder for Geography
with open('ohe_geo.pkl', 'wb') as f:
    pickle.dump(ohe_geo, f)

#### Split Data in Dependent and Independent Features

In [72]:
X = df.drop('Exited', axis=1)
y = df['Exited']

# Split Data in Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [73]:
# Save the Scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

### ANN Implementation

In [74]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

### Build ANN Model

In [75]:
# Build ANN Model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],))) ## 1st Input layer connected to 64 neurons - HL1
model.add(Dense(32, activation='relu')) # no shaoe needed here as it is a hidden layer - Hidden layer with 32 neurons - HL2
model.add(Dense(1, activation='sigmoid')) # Output layer with 1 neuron for binary classification

In [76]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 64)                832       
                                                                 
 dense_9 (Dense)             (None, 32)                2080      
                                                                 
 dense_10 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#### Model Preparation

In order to do forward and backward propagation, we need to compile the model

In [77]:
# compile the model with optimizer, loss function and metrics
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])

In [78]:
# Set up the tensorboard callback
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

#### Train the Model

In [79]:
history = model.fit(X_train_scaled, y_train,    
                    validation_data=(X_test_scaled, y_test),
                    epochs=100,
                    callbacks=[tensorboard_callback, early_stopping])

Epoch 1/100
250/250 [==============================] - 2s 7ms/step - loss: 0.5535 - accuracy: 0.7806 - val_loss: 0.5473 - val_accuracy: 0.8030
Epoch 2/100
250/250 [==============================] - 1s 6ms/step - loss: 1.2733 - accuracy: 0.7465 - val_loss: 1.9880 - val_accuracy: 0.6815
Epoch 3/100
250/250 [==============================] - 1s 6ms/step - loss: 2.8616 - accuracy: 0.7368 - val_loss: 3.8312 - val_accuracy: 0.7565
Epoch 4/100
250/250 [==============================] - 1s 6ms/step - loss: 3.3271 - accuracy: 0.7303 - val_loss: 17.8678 - val_accuracy: 0.6560
Epoch 5/100
250/250 [==============================] - 1s 6ms/step - loss: 6.7133 - accuracy: 0.7289 - val_loss: 3.3616 - val_accuracy: 0.8045
Epoch 6/100
250/250 [==============================] - 1s 6ms/step - loss: 8.6875 - accuracy: 0.7306 - val_loss: 9.0012 - val_accuracy: 0.7670
Epoch 7/100
250/250 [==============================] - 1s 6ms/step - loss: 13.4958 - accuracy: 0.7394 - val_loss: 7.7190 - val_accuracy: 0.73

In [81]:
# Save model file
model.save('customer_churn_model.h5')

In [84]:
# Load TensorBoard Extension
%load_ext tensorboard

# Launch TensorBoard
%tensorboard --logdir logs/fit

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 48125), started 0:00:59 ago. (Use '!kill 48125' to kill it.)